In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\dell\\Desktop\\Alzheimer-s-Disease-Classification'

In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/shakti-sarada/Alzheimer-s-Disease-Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "shakti-sarada"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "aa221de64fb387162131a93f20ba78393d3e12cd"

In [3]:
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from AlzheimerDiseaseClassification.constants import *
from AlzheimerDiseaseClassification.utils.common import read_yaml,create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Alzheimer_s Dataset",
            mlflow_uri="https://dagshub.com/shakti-sarada/Alzheimer-s-Disease-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config


In [7]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [8]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_score = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0],"accuracy":self.score[1]}
            )

            # Model Registry does not work with file store

            if tracking_uri_type_score != "file":
                
                # Register the Model
                # There are other ways to use the model registry, which depends on the use case,
                # Please refer to the docs for more information:
                #hhtps://mlflow.org/docs/latest/model-registry.html#api-workflow

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")

            else:
                mlflow.keras.log_model(self.model,"model")

In [9]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-02-20 04:15:01,358: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-20 04:15:01,374: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-20 04:15:01,375: INFO: common: created directory at: artifacts]
Found 1865 images belonging to 2 classes.
47/47 [==============================] - 306s 7s/step - loss: 10.4047 - accuracy: 0.7437
[2024-02-20 04:20:08,585: INFO: common: json file saved at: scores.json]


2024/02/20 04:20:10 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\dell\AppData\Local\Temp\tmp57maiu95\model\data\model\assets
[2024-02-20 04:20:12,717: INFO: builder_impl: Assets written to: C:\Users\dell\AppData\Local\Temp\tmp57maiu95\model\data\model\assets]


c:\Users\dell\anaconda3\envs\alzheimer\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/02/20 04:21:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 7
Created version '7' of model 'VGG16Model'.
